# Larger Examples

A collection of more complex examples. Note that these are here temporarily and will eventually moved to the AlgorithmX website.

## Mergesort

In [1]:
import algorithmx
import math

widget = algorithmx.jupyter_widget(buttons=True)
canvas = widget.canvas().size((650, 500))

# Change this to create a different example!
unsorted = [7, 3, 2, 6, 8, 5, 1, 4]

node_style = {
    'shape': 'rect',
    'size': (30, 15),
    'fixed': True
}

def node_pos(i, level):
    x = (i - (len(unsorted) - 1) / 2) * 80
    y = -(level - math.ceil(math.log2(len(unsorted))) / 2) * 120
    return (x, y)

def node_id(i, level):
    return str(i) + '_' + str(level)

def animate_merge_edge(node_prev, node_cur, number, color):
    edge = (node_prev, node_cur)

    canvas.node(node_prev).color(color)
    canvas.edge(edge).add().directed(True)
    canvas.pause(0.5)

    canvas.edge(edge).traverse().color(color).pause(0.2)
    canvas.node(node_cur).label().text(number).visible(True)
    canvas.pause(0.5)


def animate_merge(left, right, index, level):
    total_len = len(left) + len(right)
    prev_ids = [node_id(index + i, level - 1) for i in range(total_len)]
    cur_ids = [node_id(index + i, level) for i in range(total_len)]

    new_nodes = canvas.nodes(cur_ids).add().set(node_style)
    new_nodes.pos(lambda n, i: node_pos(index + i, level))
    new_nodes.label().visible(False)

    canvas.edges([(cur_ids[i], cur_ids[i + 1])
        for i in range(len(cur_ids) - 1)]).add()

    canvas.pause(0.8)

    li = 0
    ri = 0
    result = []
    for i in range(total_len):
        if ri == len(right) or li < len(left) and left[li] < right[ri]:
            result.append(left[li])
            animate_merge_edge(prev_ids[li], cur_ids[i],
                left[li], 'green')
            li += 1
        else:
            result.append(right[ri])
            animate_merge_edge(prev_ids[len(left) + ri], cur_ids[i],
                right[ri], 'red')
            ri += 1

    return result

def mergesort(array, index=0, level=None):
    level = math.ceil(math.log2(len(array))) if level is None else level

    if len(array) == 1:
        if level != 0:
            # Animation edge case for arrays that are not a power of 2 in length
            animate_merge(array, [], index, level)
        return array

    split_index = int(len(array) / 2)
    left = mergesort(array[0:split_index], index, level - 1)
    right = mergesort(array[split_index:], index + split_index, level - 1)

    return animate_merge(left, right, index, level)


top_ids = [node_id(i, 0) for i in range(len(unsorted))]

top_nodes = canvas.nodes(top_ids).add().set(node_style)
top_nodes.pos(lambda n, i: node_pos(i, 0))
top_nodes.data(unsorted).label().text(lambda n: n)
canvas.pause(0.5)

mergesort(unsorted)


widget

JupyterWidget()

## Maze Solving

In [2]:
import algorithmx

widget = algorithmx.jupyter_widget(buttons=True)
canvas = widget.canvas().size((500, 400))

maze = [
    'x....x..',
    '..xx...x',
    'x...xxxx',
    '.xx.x...',
    '......x.',
    'x.xxx.x.',
    'x...x.xx',
    '..x.x..x'
]
maze_width, maze_height = len(maze[0]), len(maze)
maze_path_char = '.'
maze_start, maze_end = (0, maze_height - 1), (maze_width - 1, 0)

def node_pos(coords):
    x = (coords[0] - (maze_width - 1) / 2) * 50
    y = -(coords[1] - (maze_height - 1) / 2) * 50
    return (x, y)

def get_adjacent(coords):
    x, y = coords[0], coords[1]
    adjacent = []

    if x < maze_width - 1 and maze[y][x + 1] == maze_path_char:
        adjacent.append((x + 1, y))
    if x > 0 and maze[y][x - 1] == maze_path_char:
        adjacent.append((x - 1, y))
    if y < maze_height - 1 and maze[y + 1][x] == maze_path_char:
        adjacent.append((x, y + 1))
    if y > 0 and maze[y - 1][x] == maze_path_char:
        adjacent.append((x, y - 1))

    return adjacent

def animate_dfs():
    stack = get_adjacent(maze_start)

    seen = {(x, y): False for x in range(maze_width)
        for y in range(maze_height)}

    canvas.duration(1).pan(node_pos(maze_start)).zoom(2)
    canvas.pause(1)

    prev = maze_start
    cur = None
    seen[prev] = True
    while len(stack) > 0:
        cur = stack[len(stack) - 1]
        seen[cur] = True

        if prev != maze_start:
            canvas.node(prev).color('green').pause(0.3)

        edge = (prev, cur)
        canvas.edge(edge).traverse().color('green')
        canvas.pan(node_pos(cur))
        canvas.pause(0.3)

        if cur == maze_end:
            break

        adjacent = [n for n in get_adjacent(cur) if not seen[n]]
        while len(adjacent) == 0:
            stack = stack[0:len(stack) - 1]
            prev = stack[len(stack) - 1]

            canvas.node(cur).color('red').pause(0.3)
            canvas.edge((cur, prev)).traverse().color('red')
            canvas.pan(node_pos(prev))
            canvas.pause(0.3)

            adjacent = [n for n in get_adjacent(prev) if not seen[n]]
            cur = prev

        stack.append(adjacent[0])
        prev = cur

    canvas.pause(0.5)
    canvas.duration(2).pan((0, 0)).zoom(1)

def create_graph():
    node_coords = [(x, y) for x in range(maze_width)
        for y in range(maze_height)]

    canvas.nodes(node_coords).add().fixed(True) \
        .pos(lambda n: node_pos(n)) \
        .label().text('')

    canvas.nodes([maze_start, maze_end]).duration(0).color('blue')

    edge_ids = []
    for x, y in node_coords:
        if maze[y][x] == maze_path_char:
            for x2, y2 in get_adjacent((x, y)):
                if x2 >= x and y2 >= y:
                    edge_ids.append(((x, y), (x2, y2)))

    canvas.edges(edge_ids).add()

create_graph()
canvas.pause(1)
animate_dfs()

widget

JupyterWidget()